In [1]:
import gc
import ctypes

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, f1_score
from sklearn.model_selection import GridSearchCV

from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline

from sklearn.decomposition import PCA
from umap.umap_ import UMAP
from sklearn.decomposition import FastICA
import xgboost as xgb

from sklearn.feature_selection import SelectFromModel

from multiprocessing import Process, Manager

from sklearn.cluster import KMeans



# print ("Shape of final train and test sets:", X_train_scaled.shape, X_test_scaled.shape)    

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database',)).History will not be written to the database.


Using TensorFlow backend.


### Load data with IPC approach

In [ ]:
def set_Data(data):
    ppmi = pd.read_csv('../../datasets/preprocessed/trans_processed_PPMI_data.csv')
    ppmi.rename(columns={'Unnamed: 0':'Sentrix_position'}, inplace=True)
    ppmi.set_index('Sentrix_position', inplace=True)
    ppmi = ppmi.transpose()

    encoder = LabelEncoder()
    label = encoder.fit_transform(ppmi['Category'])

    tr = ppmi.drop(['Category'], axis=1)
    X = tr.values
    y = label
    print(X.shape)
    print(y.shape)

#     print("StratifiedSampling check")
#     split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
#     split.get_n_splits(X, y)

#     for train_index, test_index in split.split(X, y):
#         X_train, X_test = X[train_index], X[test_index]
#         y_train, data['y_test'] = y[train_index], y[test_index]

#     print("Oversampling check")
#     oversampler = SMOTE(random_state=42)
#     X_sampled, data['y_sampled'] = oversampler.fit_resample(X, y)
#     print(X_sampled.shape)
    data['y'] = y
    print("Scaling check")
    scaler = StandardScaler()
#     scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    data['X_scaled_1'] = X_scaled[:300].reshape((1, -1))
    data['X_scaled_2'] = X_scaled[300:].reshape((1, -1))
    
    print("Returning check")

manager = Manager()
data = manager.dict()

print("CHECKPOINT1")
#     p = Process(target=set_Data, args=(X_train_scaled, X_test_scaled, y_train_sampled, y_test,))
p = Process(target=set_Data, args=(data,))
print("CHECKPOINT2")
p.start()
print("CHECKPOINT3")
p.join()

y = data['y']
X_scaled = np.append(data['X_scaled_1'], data['X_scaled_2']).reshape(436, 747668)

### Do clustering

In [34]:
## clustering
X_for_clustering = X_scaled.transpose()
print("Shape of X for clustering:",  X_for_clustering.shape)
### Need to tune n_clusters
# ks =  [3]
cluster_dict={}
# for k in ks:
k = 6
kmeans = KMeans(n_clusters=k, random_state=0, n_jobs=k, verbose=1).fit(X_for_clustering)
clusters = kmeans.labels_
for n in range(k):
    print("Index of features in cluster", n+1, "is:")
    i = np.where(clusters == n)
    print(i)
    print("Number of features in cluster",n+1, "=", i[0].size)
    X_cluster = X_for_clustering[i].transpose()
    print("Shape of X_train_cluster",n+1,"is", X_cluster.shape)
    cluster_dict[n] = X_cluster

Shape of X for clustering: (747668, 436)
Index of features in cluster 1 is:
(array([     5,      6,     12, ..., 747644, 747652, 747656]),)
Number of features in cluster 1 = 113388
Shape of X_train_cluster 1 is (436, 113388)
Index of features in cluster 2 is:
(array([     0,      7,     24, ..., 747636, 747638, 747655]),)
Number of features in cluster 2 = 86936
Shape of X_train_cluster 2 is (436, 86936)
Index of features in cluster 3 is:
(array([    64,     70,     75, ..., 747615, 747616, 747651]),)
Number of features in cluster 3 = 61973
Shape of X_train_cluster 3 is (436, 61973)
Index of features in cluster 4 is:
(array([     2,      4,      9, ..., 747659, 747661, 747664]),)
Number of features in cluster 4 = 190334
Shape of X_train_cluster 4 is (436, 190334)
Index of features in cluster 5 is:
(array([     1,      3,     11, ..., 747665, 747666, 747667]),)
Number of features in cluster 5 = 173700
Shape of X_train_cluster 5 is (436, 173700)
Index of features in cluster 6 is:
(array([

In [128]:
## Apply umap
cluster_reduced_dict = {}
for i in range(k):
    ###DR
#     umap = UMAP(n_components=2)
    ###FS
    sel_ = SelectFromModel(Lasso(alpha=0.08, tol=0.01, random_state=42))
    sel_.fit(cluster_dict[i], y)
#     X_reduced_cluster = umap.fit_transform(cluster_dict[i])
    X_reduced_cluster = sel_.transform(cluster_dict[i])
    cluster_reduced_dict[i] = X_reduced_cluster
print(cluster_reduced_dict)
temp = np.concatenate((cluster_reduced_dict[0], cluster_reduced_dict[1]), axis=1)
### Since only one cluster value used here, take X_reduced out
for j in range(3):
    temp =  np.concatenate((temp, cluster_reduced_dict[j+2]), axis=1)
X_reduced =  np.concatenate((temp, cluster_reduced_dict[k-1]), axis=1)
print("Final shape of input data after clustering:", X_reduced.shape)

/home/rz296/miniconda3/envs/partII/lib/python3.6/site-packages/sklearn/feature_selection/_base.py:81: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  UserWarning)


{0: array([[ 9.51976636e-01],
       [ 5.56560171e-01],
       [ 7.76275084e-01],
       [ 4.79367433e-01],
       [ 1.14142573e+00],
       [ 3.31439041e-01],
       [-1.33813024e+00],
       [-4.31586549e+00],
       [ 2.28804640e-01],
       [ 4.27779904e-01],
       [-4.20594578e-01],
       [-4.29070845e-01],
       [ 2.04329464e-01],
       [-2.12016765e-01],
       [-3.01108928e-01],
       [ 8.37033058e-01],
       [-6.54140525e-02],
       [-6.18389814e-01],
       [ 1.10329465e-01],
       [ 3.47578303e-01],
       [ 8.97194968e-01],
       [ 2.42054038e-01],
       [ 4.43030597e-01],
       [ 9.86929872e-02],
       [-2.14789024e-01],
       [ 5.13505524e-01],
       [-1.16980239e+00],
       [-9.00396669e-01],
       [ 1.17426794e+00],
       [ 1.35781164e+00],
       [-7.35756405e-01],
       [-9.43792379e-01],
       [ 9.41371327e-01],
       [-1.61231904e-01],
       [-1.22184846e+00],
       [-7.91717902e-02],
       [-4.61075215e-02],
       [-3.81486712e-01],
       [

In [129]:
### Split training and testing sets
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
split.get_n_splits(X_reduced, y)

for train_index, test_index in split.split(X_reduced, y):
    X_train, X_test = X_reduced[train_index], X_reduced[test_index]
    y_train, y_test= y[train_index], y[test_index]
    
### Upsample here
oversampler = SMOTE(random_state=42)
X_tr_sampled, y_tr_sampled = oversampler.fit_resample(X_train, y_train)

In [94]:
### Train on the concatenated clusters using XGBoost
clf = xgb.XGBClassifier(
            objective='binary:logistic', 
            seed=42, 
            tree_method='gpu_hist',
            learning_rate=0.5,
            subsample=0.3,
            gpu_id=1,
            colsample_bytree=0.4,
            n_estimators=70,
            max_depth=3
        )

clf.fit(X_train, y_train)

y_pred_tr = clf.predict(X_tr_sampled)
y_pred_te = clf.predict(X_test)
cm_tr = confusion_matrix(y_tr_sampled, y_pred_tr)
cm_te = confusion_matrix(y_test, y_pred_te)
print("Confusion matrix of clustered + reduced training set using UMAP+XGBoost:")
print(cm_tr)
print("Confusion matrix of clustered + reduced testing set using UMAP+XGBoost:")
print(cm_te)
print("precision of testing set:", precision_score(y_test, y_pred_te))


Confusion matrix of clustered + reduced training set using UMAP+XGBoost:
[[195  52]
 [  3 244]]
Confusion matrix of clustered + reduced testing set using UMAP+XGBoost:
[[13 13]
 [ 6 56]]
precision of testing set: 0.8115942028985508


In [93]:
### Train using FS + LR
clf_lr = LogisticRegression(
    max_iter=1000, 
    penalty='l2', 
    solver='saga',
    C = 1
)

clf_lr.fit(X_train, y_train)

y_pred_tr = clf_lr.predict(X_tr_sampled)
y_pred_te = clf_lr.predict(X_test)
cm_tr = confusion_matrix(y_tr_sampled, y_pred_tr)
cm_te = confusion_matrix(y_test, y_pred_te)
print("Confusion matrix of clustered + reduced training set using FS+LR:")
print(cm_tr)
print("Confusion matrix of clustered + reduced testing set using FS+LR:")
print(cm_te)
print("precision of testing set:", precision_score(y_test, y_pred_te))


Confusion matrix of clustered + reduced training set using FS+LR:
[[137 110]
 [ 19 228]]
Confusion matrix of clustered + reduced testing set using FS+LR:
[[12 14]
 [ 2 60]]
precision of testing set: 0.8108108108108109


In [131]:
#### Train using FS + SVM
clf_svm = SVC(
    max_iter=3000, 
    gamma=1.5, 
    kernel='poly', 
    tol=0.01,
    C = 0.0001,
    class_weight='balanced')

clf_svm.fit(X_train, y_train)

y_pred_tr = clf_svm.predict(X_tr_sampled)
y_pred_te = clf_svm.predict(X_test)
cm_tr = confusion_matrix(y_tr_sampled, y_pred_tr)
cm_te = confusion_matrix(y_test, y_pred_te)
print("Confusion matrix of clustered + reduced training set using FS+LR:")
print(cm_tr)
print("Confusion matrix of clustered + reduced testing set using FS+LR:")
print(cm_te)
print("precision of testing set:", precision_score(y_test, y_pred_te))


Confusion matrix of clustered + reduced training set using FS+LR:
[[237  10]
 [  0 247]]
Confusion matrix of clustered + reduced testing set using FS+LR:
[[13 13]
 [ 1 61]]
precision of testing set: 0.8243243243243243


## Clustering on PROPAG-AGEING dataset

In [2]:
import joblib

In [3]:
###Clustering on ppg
## clustering
X_scaled = np.load('../../datasets/preprocessed/npy_files/X_scaled_ppg.npy')
y = np.load('../../datasets/preprocessed/npy_files/y_ppg.npy')

In [4]:
cluster_dict={}
k=6
for n in range(k):
    print("Index of features in cluster", n+1, "is:")
    i = joblib.load("cluster_index_"+str(n)+".joblib")
    print(i)
    print("Number of features in cluster",n+1, "=", i[0].size)
    X_cluster = (X_scaled.transpose())[i].transpose()
    print("Shape of X_train_cluster",n+1,"is", X_cluster.shape)
    cluster_dict[n] = X_cluster

Index of features in cluster 1 is:
(array([     5,      6,     12, ..., 747644, 747652, 747656]),)
Number of features in cluster 1 = 113388
Shape of X_train_cluster 1 is (361, 113388)
Index of features in cluster 2 is:
(array([     0,      7,     24, ..., 747636, 747638, 747655]),)
Number of features in cluster 2 = 86936
Shape of X_train_cluster 2 is (361, 86936)
Index of features in cluster 3 is:
(array([    64,     70,     75, ..., 747615, 747616, 747651]),)
Number of features in cluster 3 = 61973
Shape of X_train_cluster 3 is (361, 61973)
Index of features in cluster 4 is:
(array([     2,      4,      9, ..., 747659, 747661, 747664]),)
Number of features in cluster 4 = 190334
Shape of X_train_cluster 4 is (361, 190334)
Index of features in cluster 5 is:
(array([     1,      3,     11, ..., 747665, 747666, 747667]),)
Number of features in cluster 5 = 173700
Shape of X_train_cluster 5 is (361, 173700)
Index of features in cluster 6 is:
(array([     8,     13,     29, ..., 747646, 7476

In [5]:
cluster_reduced_dict = {}
for i in range(k):
    sel_ = joblib.load("FS_selector_"+str(i)+".joblib")
    
    X_reduced_cluster = sel_.transform(cluster_dict[i])
    cluster_reduced_dict[i] = X_reduced_cluster

temp = np.concatenate((cluster_reduced_dict[0], cluster_reduced_dict[1]), axis=1)
### Since only one cluster value used here, take X_reduced out
for j in range(3):
    temp =  np.concatenate((temp, cluster_reduced_dict[j+2]), axis=1)
X_reduced =  np.concatenate((temp, cluster_reduced_dict[k-1]), axis=1)
print("Final shape of input data after clustering:", X_reduced.shape)


Final shape of input data after clustering: (361, 26)


/home/rz296/miniconda3/envs/partII/lib/python3.6/site-packages/sklearn/feature_selection/_base.py:81: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  UserWarning)


In [6]:
# ### Split training and testing sets
# split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
# split.get_n_splits(X_reduced, y)

# for train_index, test_index in split.split(X_reduced, y):
#     X_train, X_test = X_reduced[train_index], X_reduced[test_index]
#     y_train, y_test= y[train_index], y[test_index]
    
# ### Upsample here
# oversampler = SMOTE(random_state=42)
# X_tr_sampled, y_tr_sampled = oversampler.fit_resample(X_train, y_train)

In [ ]:
clf = joblib.load("GTB_model.joblib") 

!CUDA_VISIBLE_DEVICES=2

clf.fit(X_scaled, y)

y_pred = clf.predict(X_reduced)
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix of clustered PPG:")
print(cm)
print("precision of testing set:", precision_score(y_test, y_pred))
print("accuracy of testing set:", accuracy_score(y_test, y_pred))
print("f1 score of testing set:", f1_score(y_test, y_pred))